In [1]:
from settings import *
from init_state import read_initial_state
from helpers import timeInMinutes
import policies.hlv_master

import folium
import random

In [2]:
state = read_initial_state(sb_jsonFilename = SB_INSTANCE_FILE, ff_jsonFilename=FF_INSTANCE_FILE)

seed = random.randint(1, 3000)
state.set_seed(seed)

In [3]:
# policy = policies.hlv_master.BS_PILOT()
# state.set_sb_vehicles([policy])

policy2 = policies.hlv_master.BS_PILOT_FF()
state.set_ff_vehicles([policy2])

In [4]:
# Create a map centered around a default location
mymap = folium.Map(location=[63.446827, 10.421906], zoom_start=13)

color_scale = [
    "#0000ff", "#1600e6", "#2d00cc", "#4300b3", "#590099", "#700080",
    "#860066", "#9c004d", "#b30033", "#c9001a", "#df0000", "#ff0000"
]

# Plot the hexagons on the map
for hex in state.get_areas():
    hex_num_bikes = len(hex.get_bikes())
    # Create a polygon and add to the map
    if hex_num_bikes > 0:
        folium.Polygon(locations=hex.border_vertices, fill=True, color=color_scale[hex_num_bikes-1], weight=2, popup= str(hex_num_bikes)).add_to(mymap)

    # for bike in hex.get_bikes():
    #     folium.CircleMarker(location=[bike.get_lat(), bike.get_lon()], radius=2, color='green', popup=str(len(hex.get_bikes())), fill=True, fill_color='red').add_to(mymap)

# Plot trip coordinates as markers on the map
for station in state.get_stations():
    #folium.Marker(location=[lat, lng], popup=trip['trip_id'], icon=folium.Icon(color='blue')).add_to(mymap)
    folium.CircleMarker(location=[station.get_lat(), station.get_lon()], radius=2, color='green', fill=True, fill_color='red').add_to(mymap)

mymap

In [5]:
import sim
import demand
import target_state

tstate = target_state.HLVTargetState(FF_TARGET_STATE_FILE)
# tstate = target_state.HalfCapacityTargetState()

###############################################################################
# Set up demand

dmand = demand.demand.Demand()

###############################################################################
# Set up simulator


In [6]:
import sim
import demand
import target_state

simulator = sim.Simulator(
    initial_state = state,
    target_state = tstate,
    demand = dmand,
    start_time = timeInMinutes(hours=7),
    duration = timeInMinutes(days=2),
    verbose = True,
)

simulator.run()

Running Sim |▌                               | 2% - ETA 1s

Running Sim |████████████████████████████████| 100% - ETA 0s 


In [36]:

# Create a map centered around a default location
mymap = folium.Map(location=[63.446827, 10.421906], zoom_start=13)
print("All bikes:", len(state.get_all_ff_bikes()))

# Plot the hexagons on the map
for hex in simulator.state.get_areas():
    # Create a polygon and add to the map
    hex_num_bikes = len(hex.get_bikes())
    # Create a polygon and add to the map
    # if hex_num_bikes > 10:
    #     folium.Polygon(locations=hex.border_vertices, fill=True, color="red", weight=2, popup= hex.location_id + " " +str(hex_num_bikes)).add_to(mymap)
    # elif hex_num_bikes > 0:
    #     folium.Polygon(locations=hex.border_vertices, fill=True, color="#000000", weight=2, popup= hex.location_id + " " + str(hex_num_bikes)).add_to(mymap)
    
    # num_starvations = hex.metrics.get_aggregate_value("starvations")

    # if num_starvations > 20:
    #     folium.Polygon(locations=hex.border_vertices, fill=True, color='red', weight=2, popup= str(num_starvations)).add_to(mymap)
    # elif num_starvations > 10:
    #     folium.Polygon(locations=hex.border_vertices, fill=True, color='blue', weight=2, popup= str(num_starvations)).add_to(mymap)
    # elif num_starvations > 0:
    #     folium.Polygon(locations=hex.border_vertices, fill=True, color='black', weight=2, popup= str(num_starvations)).add_to(mymap)

    sum_mp = 0
    for day in range(7):
        for hour in range(24):
            sum_mp += hex.get_leave_intensity(day, hour)
    sum_mp = sum_mp / (7)
    if sum_mp > 5:
        folium.Polygon(locations=hex.border_vertices, fill=True, color='red', weight=2, popup= str(sum_mp)).add_to(mymap)

# Plot trip coordinates as markers on the map
# for station in simulator.state.get_stations():
#     #folium.Marker(location=[lat, lng], popup=trip['trip_id'], icon=folium.Icon(color='blue')).add_to(mymap)
#     folium.CircleMarker(location=[station.get_lat(), station.get_lon()], radius=2, color='green', fill=True, fill_color='red', popup=str(len(station.get_bikes()))).add_to(mymap)

mymap

All bikes: 1066


In [11]:
print("events", simulator.metrics.get_aggregate_value('events'))
print("trips", simulator.metrics.get_aggregate_value('trips'))
print("random trips", simulator.metrics.get_aggregate_value('random trips'))
print("escooter departure", simulator.metrics.get_aggregate_value('escooter departure'))
print("escooter arrival", simulator.metrics.get_aggregate_value('escooter arrival'))
print("vehicle arrivals", simulator.metrics.get_aggregate_value('vehicle arrivals'))

print("failed events", simulator.metrics.get_aggregate_value('failed events'))
print("escooter starvations", simulator.metrics.get_aggregate_value('escooter starvations'))
print("battery starvations", simulator.metrics.get_aggregate_value('battery starvations'))
print("battery violations", simulator.metrics.get_aggregate_value('battery violations'))
print("Number of battery swaps", simulator.metrics.get_aggregate_value('num battery swaps'))
print("Number of escooter deliveries", simulator.metrics.get_aggregate_value('num escooter deliveries'))
print("Number of escooter pickups", simulator.metrics.get_aggregate_value('num escooter pickups'))
print("Number of helping bike deliveries", simulator.metrics.get_aggregate_value('num helping bike deliveries'))
print("Number of helping bike pickups", simulator.metrics.get_aggregate_value('num helping bike pickups'))

events 29150
trips 15986
random trips 148
escooter departure 7572
escooter arrival 7519
vehicle arrivals 179
failed events 6393
escooter starvations 4974
battery starvations 0
battery violations 0
Number of battery swaps 60
Number of escooter deliveries 0
Number of escooter pickups 20
Number of helping bike deliveries 0
Number of helping bike pickups 0


In [45]:
import gzip
import json
import os
import sim

with open(f"instances/Ryde/TR_random_100.json", "r") as ff_infile:
    dirname = os.path.dirname('instances/Ryde/TR_random_100')

    # load json state
    ff_statedata = json.load(ff_infile)

state = sim.State.get_initial_ff_state(ff_statedata)

In [46]:
ff_statedata["traveltime_matrix"] = {"__".join(map(str, key)): value for key, value in state.traveltime_matrix.items() }

In [47]:
ff_statedata["traveltime_vehicle_matrix"] = {"__".join(map(str, key)): value for key, value in state.traveltime_vehicle_matrix.items() }

In [48]:
with open('instances/Ryde/TR_random_100_matrixes.json', 'w') as file:
    json.dump(ff_statedata, file, indent=4)

In [51]:
with gzip.open('instances/Ryde/TR_random_100_matrixes.json.gz', 'wt', encoding='utf-8') as file:
    json.dump(ff_statedata, file, indent=4)